## Importing the required libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


# statistical tests libraries
from scipy.stats import levene
from scipy.stats import mannwhitneyu
from statsmodels.stats.anova import anova_lm
import statsmodels.formula.api as sfa

## Load dataset

In [3]:
data= pd.read_csv("capstone_encoded.csv")

# Modeling

In [4]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [5]:
cols=['Model Name','Train R2 Score','Test R2 Score','Train RMSE','Test RMSE','MAPE']

result=pd.DataFrame(columns=cols)

### Without outlier model

In [6]:
# Removing the outliers (as we already see above in the file.)
car_wo = data.loc[data.price<57000]

In [8]:
x = car_wo.drop('price',axis =1)
y = car_wo.price

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size = .30, random_state = 42)

# Checking the shape or train and test data.
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((191600, 30), (82115, 30), (191600,), (82115,))

#### Correlation with Target

In [10]:
pd.DataFrame(car_wo.corr().iloc[1,:])

,region
Unnamed: 0,0.002896
region,1.000000
price,-0.072851
year,-0.044475
manufacturer,0.003664
model,-0.001134
condition,0.041957
cylinders,-0.011124
odometer,0.037452
size,-0.012060


### Decision Tree

In [11]:
dt=DecisionTreeRegressor(random_state = 42)


# Model fitting
model_dt = dt.fit(xtrain,ytrain)


# Model prediction
pred_test = model_dt.predict(xtest)
pred_train = model_dt.predict(xtrain)


#R2 score:
r2score_train=r2_score(ytrain,pred_train)
r2score_test=r2_score(ytest,pred_test)


# train and test rmse:
rmse_train=np.sqrt(mean_squared_error(ytrain,pred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))

# MAPE:
mape = mean_absolute_percentage_error(ytest, pred_test)

In [12]:
mlrmodel=pd.Series({'Model Name':'Decision Tree','Train R2 Score':r2score_train,'Test R2 Score':r2score_test,
                    'Train RMSE': rmse_train,'Test RMSE': rmse_test,'MAPE':mape})

result=result.append(mlrmodel,ignore_index=True)

result

,Model Name,Train R2 Score,Test R2 Score,Train RMSE,Test RMSE,MAPE
0,Decision Tree,1.0,0.698599,0.0,6791.589022,41.931885


### Ridge

In [13]:
ridge=Ridge()

# Model fitting
model_ridge = ridge.fit(xtrain,ytrain)


# Model prediction
pred_test = model_ridge.predict(xtest)
pred_train = model_ridge.predict(xtrain)


#R2 score:
r2score_train=r2_score(ytrain,pred_train)
r2score_test=r2_score(ytest,pred_test)


# train and test rmse:
rmse_train=np.sqrt(mean_squared_error(ytrain,pred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))

# MAPE:
mape = mean_absolute_percentage_error(ytest, pred_test)

In [14]:
mlrmodel=pd.Series({'Model Name':'Ridge','Train R2 Score':r2score_train,'Test R2 Score':r2score_test,
                    'Train RMSE': rmse_train,'Test RMSE': rmse_test,'MAPE':mape})

result=result.append(mlrmodel,ignore_index=True)

result

,Model Name,Train R2 Score,Test R2 Score,Train RMSE,Test RMSE,MAPE
0,Decision Tree,1.000000,0.698599,0.000000,6791.589022,41.931885
1,Ridge,0.551583,0.550598,8324.314912,8293.102361,65.793066


# 

### Lasso

In [15]:
lasso = Lasso()


# Model fitting
model_lasso = lasso.fit(xtrain,ytrain)


# Model prediction
pred_test = model_lasso.predict(xtest)
pred_train = model_lasso.predict(xtrain)


#R2 score:
r2score_train=r2_score(ytrain,pred_train)
r2score_test=r2_score(ytest,pred_test)


# train and test rmse:
rmse_train=np.sqrt(mean_squared_error(ytrain,pred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))

# MAPE:
mape = mean_absolute_percentage_error(ytest, pred_test)

In [16]:
mlrmodel=pd.Series({'Model Name':'Lasso','Train R2 Score':r2score_train,'Test R2 Score':r2score_test,
                    'Train RMSE': rmse_train,'Test RMSE': rmse_test,'MAPE':mape})

result=result.append(mlrmodel,ignore_index=True)

result

,Model Name,Train R2 Score,Test R2 Score,Train RMSE,Test RMSE,MAPE
0,Decision Tree,1.000000,0.698599,0.000000,6791.589022,41.931885
1,Ridge,0.551583,0.550598,8324.314912,8293.102361,65.793066
2,Lasso,0.551568,0.550605,8324.456660,8293.032234,65.761432


# 

### Elastic Net

In [17]:
en = ElasticNet()


# Model fitting
model_en = en.fit(xtrain,ytrain)


# Model prediction
pred_test = model_en.predict(xtest)
pred_train = model_en.predict(xtrain)


#R2 score:
r2score_train=r2_score(ytrain,pred_train)
r2score_test=r2_score(ytest,pred_test)


# train and test rmse:
rmse_train=np.sqrt(mean_squared_error(ytrain,pred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))

# MAPE:
mape = mean_absolute_percentage_error(ytest, pred_test)

In [18]:
mlrmodel=pd.Series({'Model Name':'Elastic Net','Train R2 Score':r2score_train,'Test R2 Score':r2score_test,
                    'Train RMSE': rmse_train,'Test RMSE': rmse_test,'MAPE':mape})

result=result.append(mlrmodel,ignore_index=True)

result

,Model Name,Train R2 Score,Test R2 Score,Train RMSE,Test RMSE,MAPE
0,Decision Tree,1.000000,0.698599,0.000000,6791.589022,41.931885
1,Ridge,0.551583,0.550598,8324.314912,8293.102361,65.793066
2,Lasso,0.551568,0.550605,8324.456660,8293.032234,65.761432
3,Elastic Net,0.420095,0.421313,9466.420771,9410.684276,62.153973


# 

### Random Forest

In [19]:
rf = RandomForestRegressor(random_state=0)


# Model fitting
model_rf = rf.fit(xtrain,ytrain)


# Model prediction
pred_test = model_rf.predict(xtest)
pred_train = model_rf.predict(xtrain)


#R2 score:
r2score_train=r2_score(ytrain,pred_train)
r2score_test=r2_score(ytest,pred_test)


# train and test rmse:
rmse_train=np.sqrt(mean_squared_error(ytrain,pred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))

# MAPE:
mape = mean_absolute_percentage_error(ytest, pred_test)

In [20]:
mlrmodel=pd.Series({'Model Name':'Random Forest','Train R2 Score':r2score_train,'Test R2 Score':r2score_test,
                    'Train RMSE': rmse_train,'Test RMSE': rmse_test,'MAPE':mape})

result=result.append(mlrmodel,ignore_index=True)

result

,Model Name,Train R2 Score,Test R2 Score,Train RMSE,Test RMSE,MAPE
0,Decision Tree,1.000000,0.698599,0.000000,6791.589022,41.931885
1,Ridge,0.551583,0.550598,8324.314912,8293.102361,65.793066
2,Lasso,0.551568,0.550605,8324.456660,8293.032234,65.761432
3,Elastic Net,0.420095,0.421313,9466.420771,9410.684276,62.153973
4,Random Forest,0.978810,0.852304,1809.561929,4754.267106,43.121343


# 

### AdaBoost Regressor

In [21]:
ada = AdaBoostRegressor(random_state=0)


# Model fitting
model_ada = ada.fit(xtrain,ytrain)


# Model prediction
pred_test = model_ada.predict(xtest)
pred_train = model_ada.predict(xtrain)


#R2 score:
r2score_train=r2_score(ytrain,pred_train)
r2score_test=r2_score(ytest,pred_test)


# train and test rmse:
rmse_train=np.sqrt(mean_squared_error(ytrain,pred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))

# MAPE:
mape = mean_absolute_percentage_error(ytest, pred_test)

In [22]:
mlrmodel=pd.Series({'Model Name':'AdaBoost','Train R2 Score':r2score_train,'Test R2 Score':r2score_test,
                    'Train RMSE': rmse_train,'Test RMSE': rmse_test,'MAPE':mape})

result=result.append(mlrmodel,ignore_index=True)

result

,Model Name,Train R2 Score,Test R2 Score,Train RMSE,Test RMSE,MAPE
0,Decision Tree,1.000000,0.698599,0.000000,6791.589022,41.931885
1,Ridge,0.551583,0.550598,8324.314912,8293.102361,65.793066
2,Lasso,0.551568,0.550605,8324.456660,8293.032234,65.761432
3,Elastic Net,0.420095,0.421313,9466.420771,9410.684276,62.153973
4,Random Forest,0.978810,0.852304,1809.561929,4754.267106,43.121343
5,AdaBoost,0.631736,0.632103,7543.748157,7503.471796,60.270101


# 

### Gradient Boosting Regressor

In [23]:
gbm = GradientBoostingRegressor(random_state=0)


# Model fitting
model_gbm = gbm.fit(xtrain,ytrain)


# Model prediction
pred_test = model_gbm.predict(xtest)
pred_train = model_gbm.predict(xtrain)


#R2 score:
r2score_train=r2_score(ytrain,pred_train)
r2score_test=r2_score(ytest,pred_test)


# train and test rmse:
rmse_train=np.sqrt(mean_squared_error(ytrain,pred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))

# MAPE:
mape = mean_absolute_percentage_error(ytest, pred_test)

In [24]:
mlrmodel=pd.Series({'Model Name':'Gradient Boost','Train R2 Score':r2score_train,'Test R2 Score':r2score_test,
                    'Train RMSE': rmse_train,'Test RMSE': rmse_test,'MAPE':mape})

result=result.append(mlrmodel,ignore_index=True)

result

,Model Name,Train R2 Score,Test R2 Score,Train RMSE,Test RMSE,MAPE
0,Decision Tree,1.000000,0.698599,0.000000,6791.589022,41.931885
1,Ridge,0.551583,0.550598,8324.314912,8293.102361,65.793066
2,Lasso,0.551568,0.550605,8324.456660,8293.032234,65.761432
3,Elastic Net,0.420095,0.421313,9466.420771,9410.684276,62.153973
4,Random Forest,0.978810,0.852304,1809.561929,4754.267106,43.121343
5,AdaBoost,0.631736,0.632103,7543.748157,7503.471796,60.270101
6,Gradient Boost,0.782004,0.784203,5804.057717,5746.743508,50.892615


# 

### XGBoost Regressor

In [25]:
xgb = XGBRegressor()


# Model fitting
model_xgb = xgb.fit(xtrain,ytrain)


# Model prediction
pred_test = model_xgb.predict(xtest)
pred_train = model_xgb.predict(xtrain)


#R2 score:
r2score_train=r2_score(ytrain,pred_train)
r2score_test=r2_score(ytest,pred_test)


# train and test rmse:
rmse_train=np.sqrt(mean_squared_error(ytrain,pred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))

# MAPE:
mape = mean_absolute_percentage_error(ytest, pred_test)

In [26]:
mlrmodel=pd.Series({'Model Name':'XGBoost','Train R2 Score':r2score_train,'Test R2 Score':r2score_test,
                    'Train RMSE': rmse_train,'Test RMSE': rmse_test,'MAPE':mape})

result=result.append(mlrmodel,ignore_index=True)

result

,Model Name,Train R2 Score,Test R2 Score,Train RMSE,Test RMSE,MAPE
0,Decision Tree,1.000000,0.698599,0.000000,6791.589022,41.931885
1,Ridge,0.551583,0.550598,8324.314912,8293.102361,65.793066
2,Lasso,0.551568,0.550605,8324.456660,8293.032234,65.761432
3,Elastic Net,0.420095,0.421313,9466.420771,9410.684276,62.153973
4,Random Forest,0.978810,0.852304,1809.561929,4754.267106,43.121343
5,AdaBoost,0.631736,0.632103,7543.748157,7503.471796,60.270101
6,Gradient Boost,0.782004,0.784203,5804.057717,5746.743508,50.892615
7,XGBoost,0.860786,0.840426,4638.193758,4941.745539,42.187473


# 

### CatBoost Regressor

In [27]:
cat = CatBoostRegressor(random_state=0)


# Model fitting
model_cat = cat.fit(xtrain,ytrain)


# Model prediction
pred_test = model_cat.predict(xtest)
pred_train = model_cat.predict(xtrain)


#R2 score:
r2score_train=r2_score(ytrain,pred_train)
r2score_test=r2_score(ytest,pred_test)


# train and test rmse:
rmse_train=np.sqrt(mean_squared_error(ytrain,pred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))

# MAPE:
mape = mean_absolute_percentage_error(ytest, pred_test)

Learning rate set to 0.093929
0:	learn: 11695.1544123	total: 185ms	remaining: 3m 4s
1:	learn: 11044.6426148	total: 218ms	remaining: 1m 48s
2:	learn: 10463.5435686	total: 248ms	remaining: 1m 22s
3:	learn: 9935.2167650	total: 277ms	remaining: 1m 8s
4:	learn: 9470.6999394	total: 305ms	remaining: 1m
5:	learn: 9078.0811450	total: 331ms	remaining: 54.9s
6:	learn: 8714.8377144	total: 359ms	remaining: 50.9s
7:	learn: 8406.8216915	total: 387ms	remaining: 48s
8:	learn: 8129.1124937	total: 413ms	remaining: 45.5s
9:	learn: 7888.9025177	total: 440ms	remaining: 43.5s
10:	learn: 7678.3971934	total: 468ms	remaining: 42.1s
11:	learn: 7481.9875711	total: 499ms	remaining: 41.1s
12:	learn: 7322.8176286	total: 526ms	remaining: 39.9s
13:	learn: 7175.5257818	total: 554ms	remaining: 39s
14:	learn: 7048.9441939	total: 584ms	remaining: 38.3s
15:	learn: 6935.8339255	total: 612ms	remaining: 37.6s
16:	learn: 6839.6334810	total: 639ms	remaining: 36.9s
17:	learn: 6752.7067400	total: 664ms	remaining: 36.2s
18:	learn:

154:	learn: 5475.1561891	total: 4.29s	remaining: 23.4s
155:	learn: 5472.2645014	total: 4.32s	remaining: 23.4s
156:	learn: 5470.5711737	total: 4.35s	remaining: 23.4s
157:	learn: 5468.8500791	total: 4.37s	remaining: 23.3s
158:	learn: 5465.7751246	total: 4.39s	remaining: 23.2s
159:	learn: 5462.1159269	total: 4.42s	remaining: 23.2s
160:	learn: 5460.2695782	total: 4.46s	remaining: 23.3s
161:	learn: 5458.0312745	total: 4.49s	remaining: 23.2s
162:	learn: 5456.1582349	total: 4.52s	remaining: 23.2s
163:	learn: 5453.7568324	total: 4.55s	remaining: 23.2s
164:	learn: 5450.1598543	total: 4.58s	remaining: 23.2s
165:	learn: 5448.1739356	total: 4.61s	remaining: 23.1s
166:	learn: 5446.8439165	total: 4.63s	remaining: 23.1s
167:	learn: 5444.8650357	total: 4.68s	remaining: 23.2s
168:	learn: 5442.2705416	total: 4.72s	remaining: 23.2s
169:	learn: 5439.9967014	total: 4.75s	remaining: 23.2s
170:	learn: 5437.8159560	total: 4.78s	remaining: 23.2s
171:	learn: 5436.7651730	total: 4.8s	remaining: 23.1s
172:	learn:

311:	learn: 5216.9913694	total: 8.34s	remaining: 18.4s
312:	learn: 5214.7642339	total: 8.37s	remaining: 18.4s
313:	learn: 5214.1652362	total: 8.39s	remaining: 18.3s
314:	learn: 5213.5248790	total: 8.42s	remaining: 18.3s
315:	learn: 5212.2890058	total: 8.45s	remaining: 18.3s
316:	learn: 5211.0423047	total: 8.47s	remaining: 18.3s
317:	learn: 5209.7105605	total: 8.5s	remaining: 18.2s
318:	learn: 5208.6330632	total: 8.53s	remaining: 18.2s
319:	learn: 5207.5321244	total: 8.56s	remaining: 18.2s
320:	learn: 5206.6733515	total: 8.59s	remaining: 18.2s
321:	learn: 5205.7789478	total: 8.61s	remaining: 18.1s
322:	learn: 5204.4481469	total: 8.64s	remaining: 18.1s
323:	learn: 5202.9819588	total: 8.67s	remaining: 18.1s
324:	learn: 5202.0314334	total: 8.7s	remaining: 18.1s
325:	learn: 5200.2360280	total: 8.73s	remaining: 18s
326:	learn: 5198.2054671	total: 8.76s	remaining: 18s
327:	learn: 5197.1453704	total: 8.79s	remaining: 18s
328:	learn: 5195.4133785	total: 8.82s	remaining: 18s
329:	learn: 5194.132

464:	learn: 5069.5515186	total: 12s	remaining: 13.8s
465:	learn: 5068.9858204	total: 12s	remaining: 13.7s
466:	learn: 5068.2681057	total: 12s	remaining: 13.7s
467:	learn: 5067.6825563	total: 12s	remaining: 13.7s
468:	learn: 5066.9139125	total: 12.1s	remaining: 13.7s
469:	learn: 5066.4295252	total: 12.1s	remaining: 13.6s
470:	learn: 5065.9299527	total: 12.1s	remaining: 13.6s
471:	learn: 5065.4679760	total: 12.1s	remaining: 13.6s
472:	learn: 5065.0984209	total: 12.2s	remaining: 13.5s
473:	learn: 5064.2998946	total: 12.2s	remaining: 13.5s
474:	learn: 5063.4658416	total: 12.2s	remaining: 13.5s
475:	learn: 5062.7792010	total: 12.2s	remaining: 13.5s
476:	learn: 5062.3417620	total: 12.2s	remaining: 13.4s
477:	learn: 5061.3236345	total: 12.3s	remaining: 13.4s
478:	learn: 5060.3504463	total: 12.3s	remaining: 13.4s
479:	learn: 5059.4223334	total: 12.3s	remaining: 13.3s
480:	learn: 5058.7117274	total: 12.3s	remaining: 13.3s
481:	learn: 5058.0180486	total: 12.4s	remaining: 13.3s
482:	learn: 5057.3

623:	learn: 4965.3202569	total: 15.7s	remaining: 9.48s
624:	learn: 4964.9704393	total: 15.8s	remaining: 9.46s
625:	learn: 4964.6058940	total: 15.8s	remaining: 9.43s
626:	learn: 4963.5671847	total: 15.8s	remaining: 9.4s
627:	learn: 4962.7410318	total: 15.8s	remaining: 9.38s
628:	learn: 4962.1189673	total: 15.8s	remaining: 9.35s
629:	learn: 4961.3203735	total: 15.9s	remaining: 9.32s
630:	learn: 4960.7028518	total: 15.9s	remaining: 9.29s
631:	learn: 4959.7060427	total: 15.9s	remaining: 9.27s
632:	learn: 4959.3566324	total: 15.9s	remaining: 9.24s
633:	learn: 4958.7896019	total: 16s	remaining: 9.21s
634:	learn: 4958.1883945	total: 16s	remaining: 9.19s
635:	learn: 4957.1005927	total: 16s	remaining: 9.16s
636:	learn: 4956.3812966	total: 16s	remaining: 9.13s
637:	learn: 4955.7719083	total: 16s	remaining: 9.1s
638:	learn: 4955.3829613	total: 16.1s	remaining: 9.08s
639:	learn: 4954.8120579	total: 16.1s	remaining: 9.05s
640:	learn: 4954.0307248	total: 16.1s	remaining: 9.03s
641:	learn: 4953.70817

782:	learn: 4880.6990260	total: 19.5s	remaining: 5.41s
783:	learn: 4880.1407964	total: 19.6s	remaining: 5.39s
784:	learn: 4879.4211863	total: 19.6s	remaining: 5.36s
785:	learn: 4878.8511239	total: 19.6s	remaining: 5.34s
786:	learn: 4878.4505862	total: 19.6s	remaining: 5.31s
787:	learn: 4877.7008467	total: 19.6s	remaining: 5.29s
788:	learn: 4877.0768287	total: 19.7s	remaining: 5.26s
789:	learn: 4876.7977163	total: 19.7s	remaining: 5.23s
790:	learn: 4876.2676925	total: 19.7s	remaining: 5.21s
791:	learn: 4875.8907591	total: 19.7s	remaining: 5.18s
792:	learn: 4875.3593126	total: 19.8s	remaining: 5.16s
793:	learn: 4875.0514932	total: 19.8s	remaining: 5.13s
794:	learn: 4874.6307499	total: 19.8s	remaining: 5.11s
795:	learn: 4874.2125217	total: 19.8s	remaining: 5.08s
796:	learn: 4873.8641456	total: 19.8s	remaining: 5.05s
797:	learn: 4873.2767050	total: 19.9s	remaining: 5.03s
798:	learn: 4872.8673145	total: 19.9s	remaining: 5s
799:	learn: 4872.5870785	total: 19.9s	remaining: 4.98s
800:	learn: 4

932:	learn: 4813.9536797	total: 23s	remaining: 1.65s
933:	learn: 4813.5500952	total: 23s	remaining: 1.63s
934:	learn: 4812.9230616	total: 23.1s	remaining: 1.6s
935:	learn: 4812.5935461	total: 23.1s	remaining: 1.58s
936:	learn: 4812.1350755	total: 23.1s	remaining: 1.55s
937:	learn: 4811.7729318	total: 23.1s	remaining: 1.53s
938:	learn: 4811.1913590	total: 23.2s	remaining: 1.5s
939:	learn: 4810.6530619	total: 23.2s	remaining: 1.48s
940:	learn: 4810.1795523	total: 23.2s	remaining: 1.46s
941:	learn: 4809.7027893	total: 23.2s	remaining: 1.43s
942:	learn: 4809.3501903	total: 23.3s	remaining: 1.41s
943:	learn: 4809.1103791	total: 23.3s	remaining: 1.38s
944:	learn: 4808.8782826	total: 23.3s	remaining: 1.36s
945:	learn: 4808.5254429	total: 23.3s	remaining: 1.33s
946:	learn: 4808.2453140	total: 23.3s	remaining: 1.31s
947:	learn: 4808.0316497	total: 23.4s	remaining: 1.28s
948:	learn: 4807.6472852	total: 23.4s	remaining: 1.26s
949:	learn: 4807.0170323	total: 23.4s	remaining: 1.23s
950:	learn: 4806

In [28]:
mlrmodel=pd.Series({'Model Name':'CatBoost','Train R2 Score':r2score_train,'Test R2 Score':r2score_test,
                    'Train RMSE': rmse_train,'Test RMSE': rmse_test,'MAPE':mape})

result=result.append(mlrmodel,ignore_index=True)

result

,Model Name,Train R2 Score,Test R2 Score,Train RMSE,Test RMSE,MAPE
0,Decision Tree,1.000000,0.698599,0.000000,6791.589022,41.931885
1,Ridge,0.551583,0.550598,8324.314912,8293.102361,65.793066
2,Lasso,0.551568,0.550605,8324.456660,8293.032234,65.761432
3,Elastic Net,0.420095,0.421313,9466.420771,9410.684276,62.153973
4,Random Forest,0.978810,0.852304,1809.561929,4754.267106,43.121343
5,AdaBoost,0.631736,0.632103,7543.748157,7503.471796,60.270101
6,Gradient Boost,0.782004,0.784203,5804.057717,5746.743508,50.892615
7,XGBoost,0.860786,0.840426,4638.193758,4941.745539,42.187473
8,CatBoost,0.851805,0.840915,4785.464446,4934.172983,43.649447


# 

##### Parameter tuning for Catboost regressor

In [29]:
param_grid = {'learning_rate': [0.09,0.1,0.2,0.3,0.4,0.5]}


grid= GridSearchCV(estimator=cat, param_grid=param_grid, scoring='neg_mean_squared_error' ,cv=3)
grid.fit(xtrain,ytrain)

0:	learn: 11709.8309932	total: 19.9ms	remaining: 19.9s
1:	learn: 11076.1708032	total: 40.9ms	remaining: 20.4s
2:	learn: 10503.2180170	total: 60.9ms	remaining: 20.2s
3:	learn: 10004.0194351	total: 78.9ms	remaining: 19.6s
4:	learn: 9552.4379961	total: 95.4ms	remaining: 19s
5:	learn: 9147.4051067	total: 113ms	remaining: 18.7s
6:	learn: 8799.2176477	total: 131ms	remaining: 18.6s
7:	learn: 8488.2744309	total: 152ms	remaining: 18.9s
8:	learn: 8208.8320644	total: 171ms	remaining: 18.9s
9:	learn: 7963.7683768	total: 189ms	remaining: 18.7s
10:	learn: 7749.0675910	total: 208ms	remaining: 18.7s
11:	learn: 7567.2364037	total: 226ms	remaining: 18.6s
12:	learn: 7398.6310155	total: 246ms	remaining: 18.7s
13:	learn: 7248.3277178	total: 264ms	remaining: 18.6s
14:	learn: 7119.7777480	total: 279ms	remaining: 18.3s
15:	learn: 6993.6914745	total: 295ms	remaining: 18.1s
16:	learn: 6888.7905639	total: 311ms	remaining: 18s
17:	learn: 6800.4881878	total: 328ms	remaining: 17.9s
18:	learn: 6713.1130046	total: 34

163:	learn: 5435.3635234	total: 2.68s	remaining: 13.7s
164:	learn: 5434.1096151	total: 2.69s	remaining: 13.6s
165:	learn: 5431.9963974	total: 2.71s	remaining: 13.6s
166:	learn: 5429.5101479	total: 2.73s	remaining: 13.6s
167:	learn: 5428.1798048	total: 2.74s	remaining: 13.6s
168:	learn: 5426.9597808	total: 2.76s	remaining: 13.6s
169:	learn: 5424.6233423	total: 2.78s	remaining: 13.6s
170:	learn: 5421.6119043	total: 2.8s	remaining: 13.6s
171:	learn: 5418.6730248	total: 2.81s	remaining: 13.5s
172:	learn: 5416.3179992	total: 2.83s	remaining: 13.5s
173:	learn: 5414.4043099	total: 2.85s	remaining: 13.5s
174:	learn: 5411.6522551	total: 2.87s	remaining: 13.5s
175:	learn: 5409.9456462	total: 2.89s	remaining: 13.5s
176:	learn: 5408.1361687	total: 2.9s	remaining: 13.5s
177:	learn: 5405.8566637	total: 2.92s	remaining: 13.5s
178:	learn: 5404.5420640	total: 2.94s	remaining: 13.5s
179:	learn: 5401.7284929	total: 2.96s	remaining: 13.5s
180:	learn: 5399.7287952	total: 2.97s	remaining: 13.5s
181:	learn: 

316:	learn: 5187.6914039	total: 5.19s	remaining: 11.2s
317:	learn: 5186.5776007	total: 5.2s	remaining: 11.2s
318:	learn: 5185.0618862	total: 5.23s	remaining: 11.2s
319:	learn: 5183.9113876	total: 5.25s	remaining: 11.1s
320:	learn: 5183.2456831	total: 5.26s	remaining: 11.1s
321:	learn: 5182.3435217	total: 5.28s	remaining: 11.1s
322:	learn: 5181.1622157	total: 5.3s	remaining: 11.1s
323:	learn: 5177.5653714	total: 5.32s	remaining: 11.1s
324:	learn: 5176.3207658	total: 5.34s	remaining: 11.1s
325:	learn: 5175.6983426	total: 5.36s	remaining: 11.1s
326:	learn: 5174.7241384	total: 5.38s	remaining: 11.1s
327:	learn: 5173.6650611	total: 5.39s	remaining: 11.1s
328:	learn: 5172.4139042	total: 5.41s	remaining: 11s
329:	learn: 5171.1398355	total: 5.43s	remaining: 11s
330:	learn: 5169.7596338	total: 5.45s	remaining: 11s
331:	learn: 5168.3244941	total: 5.47s	remaining: 11s
332:	learn: 5167.3276340	total: 5.49s	remaining: 11s
333:	learn: 5165.2189865	total: 5.51s	remaining: 11s
334:	learn: 5164.2874608

475:	learn: 5022.8362539	total: 7.85s	remaining: 8.64s
476:	learn: 5022.0400975	total: 7.87s	remaining: 8.63s
477:	learn: 5021.5223536	total: 7.88s	remaining: 8.61s
478:	learn: 5020.9928008	total: 7.9s	remaining: 8.59s
479:	learn: 5020.3315282	total: 7.92s	remaining: 8.58s
480:	learn: 5018.3763223	total: 7.93s	remaining: 8.56s
481:	learn: 5017.1605171	total: 7.95s	remaining: 8.55s
482:	learn: 5016.0525756	total: 7.97s	remaining: 8.53s
483:	learn: 5015.3985944	total: 7.99s	remaining: 8.52s
484:	learn: 5014.8567431	total: 8s	remaining: 8.5s
485:	learn: 5013.8149904	total: 8.02s	remaining: 8.48s
486:	learn: 5013.1887393	total: 8.04s	remaining: 8.47s
487:	learn: 5012.3773944	total: 8.05s	remaining: 8.45s
488:	learn: 5011.5832461	total: 8.07s	remaining: 8.44s
489:	learn: 5010.7238675	total: 8.09s	remaining: 8.42s
490:	learn: 5009.8379510	total: 8.12s	remaining: 8.41s
491:	learn: 5009.0179343	total: 8.14s	remaining: 8.4s
492:	learn: 5008.6065577	total: 8.16s	remaining: 8.39s
493:	learn: 5007

637:	learn: 4910.5441675	total: 10.5s	remaining: 5.98s
638:	learn: 4908.7247616	total: 10.6s	remaining: 5.96s
639:	learn: 4907.7417925	total: 10.6s	remaining: 5.95s
640:	learn: 4906.9920115	total: 10.6s	remaining: 5.93s
641:	learn: 4906.3957693	total: 10.6s	remaining: 5.91s
642:	learn: 4905.3991156	total: 10.6s	remaining: 5.9s
643:	learn: 4904.9462110	total: 10.6s	remaining: 5.88s
644:	learn: 4904.1844452	total: 10.7s	remaining: 5.86s
645:	learn: 4903.7613141	total: 10.7s	remaining: 5.84s
646:	learn: 4902.8730777	total: 10.7s	remaining: 5.83s
647:	learn: 4902.1924454	total: 10.7s	remaining: 5.81s
648:	learn: 4901.6170679	total: 10.7s	remaining: 5.79s
649:	learn: 4901.1158177	total: 10.7s	remaining: 5.77s
650:	learn: 4900.4428268	total: 10.7s	remaining: 5.76s
651:	learn: 4899.8793168	total: 10.8s	remaining: 5.74s
652:	learn: 4899.4883651	total: 10.8s	remaining: 5.72s
653:	learn: 4898.3881817	total: 10.8s	remaining: 5.71s
654:	learn: 4897.9101330	total: 10.8s	remaining: 5.69s
655:	learn:

...

532:	learn: 4581.8877250	total: 12.5s	remaining: 11s
533:	learn: 4581.3068651	total: 12.5s	remaining: 10.9s
534:	learn: 4580.7202599	total: 12.6s	remaining: 10.9s
535:	learn: 4579.7031419	total: 12.6s	remaining: 10.9s
536:	learn: 4579.1319781	total: 12.6s	remaining: 10.9s
537:	learn: 4578.5391408	total: 12.6s	remaining: 10.8s
538:	learn: 4577.6625966	total: 12.6s	remaining: 10.8s
539:	learn: 4576.5120653	total: 12.7s	remaining: 10.8s
540:	learn: 4575.5174390	total: 12.7s	remaining: 10.8s
541:	learn: 4574.6230673	total: 12.7s	remaining: 10.7s
542:	learn: 4573.9071711	total: 12.7s	remaining: 10.7s
543:	learn: 4572.9143062	total: 12.8s	remaining: 10.7s
544:	learn: 4572.1135022	total: 12.8s	remaining: 10.7s
545:	learn: 4571.6053534	total: 12.8s	remaining: 10.6s
546:	learn: 4570.8295173	total: 12.8s	remaining: 10.6s
547:	learn: 4569.9957403	total: 12.8s	remaining: 10.6s
548:	learn: 4569.6794252	total: 12.9s	remaining: 10.6s
549:	learn: 4569.0721546	total: 12.9s	remaining: 10.5s
550:	learn: 

687:	learn: 4457.6731034	total: 16.1s	remaining: 7.32s
688:	learn: 4457.2681519	total: 16.2s	remaining: 7.29s
689:	learn: 4456.2829578	total: 16.2s	remaining: 7.27s
690:	learn: 4455.5214607	total: 16.2s	remaining: 7.25s
691:	learn: 4454.8255196	total: 16.2s	remaining: 7.22s
692:	learn: 4454.1785501	total: 16.3s	remaining: 7.2s
693:	learn: 4453.6237136	total: 16.3s	remaining: 7.18s
694:	learn: 4453.1671718	total: 16.3s	remaining: 7.15s
695:	learn: 4452.0311521	total: 16.3s	remaining: 7.13s
696:	learn: 4451.3527014	total: 16.3s	remaining: 7.11s
697:	learn: 4450.2188077	total: 16.4s	remaining: 7.08s
698:	learn: 4449.5628993	total: 16.4s	remaining: 7.06s
699:	learn: 4448.6700154	total: 16.4s	remaining: 7.04s
700:	learn: 4448.1653721	total: 16.4s	remaining: 7.01s
701:	learn: 4447.0734109	total: 16.5s	remaining: 6.99s
702:	learn: 4446.2337589	total: 16.5s	remaining: 6.97s
703:	learn: 4445.8999572	total: 16.5s	remaining: 6.94s
704:	learn: 4444.6587099	total: 16.5s	remaining: 6.92s
705:	learn:

844:	learn: 4348.2468450	total: 19.7s	remaining: 3.62s
845:	learn: 4347.9381884	total: 19.7s	remaining: 3.59s
846:	learn: 4347.3272527	total: 19.8s	remaining: 3.57s
847:	learn: 4346.9654127	total: 19.8s	remaining: 3.55s
848:	learn: 4346.1441683	total: 19.8s	remaining: 3.52s
849:	learn: 4345.2364796	total: 19.9s	remaining: 3.5s
850:	learn: 4344.8092036	total: 19.9s	remaining: 3.48s
851:	learn: 4344.2166136	total: 19.9s	remaining: 3.46s
852:	learn: 4343.6199742	total: 19.9s	remaining: 3.43s
853:	learn: 4343.3356645	total: 19.9s	remaining: 3.41s
854:	learn: 4342.7429750	total: 20s	remaining: 3.39s
855:	learn: 4342.3817426	total: 20s	remaining: 3.36s
856:	learn: 4341.8736956	total: 20s	remaining: 3.34s
857:	learn: 4341.1782322	total: 20s	remaining: 3.32s
858:	learn: 4340.8939141	total: 20.1s	remaining: 3.29s
859:	learn: 4340.0689017	total: 20.1s	remaining: 3.27s
860:	learn: 4339.8468381	total: 20.1s	remaining: 3.25s
861:	learn: 4339.4195010	total: 20.1s	remaining: 3.22s
862:	learn: 4338.19

999:	learn: 4257.5999238	total: 23.3s	remaining: 0us


GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostRegressor object at 0x000001F2825562B0>,
             param_grid={'learning_rate': [0.09, 0.1, 0.2, 0.3, 0.4, 0.5]},
             scoring='neg_mean_squared_error')

In [30]:
grid.best_params_

{'learning_rate': 0.3}

In [31]:
cat_tune = CatBoostRegressor(learning_rate=0.3,random_state=0)

# Model fitting
model_cat_tune = cat_tune.fit(xtrain,ytrain)

# Model Prediction 
pred_test = model_cat_tune.predict(xtest)
pred_train = model_cat_tune.predict(xtrain)


#R2 score:
r2score_train=r2_score(ytrain,pred_train)
r2score_test=r2_score(ytest,pred_test)


# train and test rmse:
rmse_train=np.sqrt(mean_squared_error(ytrain,pred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))

# MAPE:
mape = mean_absolute_percentage_error(ytest, pred_test)

0:	learn: 10195.7295530	total: 36.1ms	remaining: 36.1s
1:	learn: 8729.9520141	total: 66.7ms	remaining: 33.3s
2:	learn: 7762.3573887	total: 96.8ms	remaining: 32.2s
3:	learn: 7158.2978964	total: 122ms	remaining: 30.4s
4:	learn: 6790.7204371	total: 151ms	remaining: 30.1s
5:	learn: 6566.8874708	total: 179ms	remaining: 29.6s
6:	learn: 6394.3407057	total: 205ms	remaining: 29s
7:	learn: 6280.7955483	total: 231ms	remaining: 28.6s
8:	learn: 6198.6134408	total: 256ms	remaining: 28.2s
9:	learn: 6137.8694914	total: 281ms	remaining: 27.8s
10:	learn: 6086.4512069	total: 322ms	remaining: 28.9s
11:	learn: 6043.2469929	total: 347ms	remaining: 28.6s
12:	learn: 6007.6639992	total: 374ms	remaining: 28.4s
13:	learn: 5968.1939986	total: 399ms	remaining: 28.1s
14:	learn: 5944.6085819	total: 427ms	remaining: 28s
15:	learn: 5910.0418363	total: 451ms	remaining: 27.7s
16:	learn: 5891.6764839	total: 480ms	remaining: 27.7s
17:	learn: 5875.5799137	total: 506ms	remaining: 27.6s
18:	learn: 5856.0402986	total: 535ms	r

158:	learn: 5079.0245873	total: 3.8s	remaining: 20.1s
159:	learn: 5076.8695331	total: 3.82s	remaining: 20.1s
160:	learn: 5074.7168595	total: 3.85s	remaining: 20s
161:	learn: 5071.6744461	total: 3.87s	remaining: 20s
162:	learn: 5069.6346311	total: 3.9s	remaining: 20s
163:	learn: 5068.0445293	total: 3.92s	remaining: 20s
164:	learn: 5066.6021504	total: 3.94s	remaining: 19.9s
165:	learn: 5063.8543417	total: 3.96s	remaining: 19.9s
166:	learn: 5063.0330695	total: 3.98s	remaining: 19.9s
167:	learn: 5061.2855003	total: 4.01s	remaining: 19.8s
168:	learn: 5058.7477784	total: 4.03s	remaining: 19.8s
169:	learn: 5057.0255379	total: 4.06s	remaining: 19.8s
170:	learn: 5054.6876582	total: 4.08s	remaining: 19.8s
171:	learn: 5050.5636313	total: 4.11s	remaining: 19.8s
172:	learn: 5048.3990154	total: 4.13s	remaining: 19.7s
173:	learn: 5045.6056241	total: 4.16s	remaining: 19.7s
174:	learn: 5044.0928636	total: 4.18s	remaining: 19.7s
175:	learn: 5042.1722203	total: 4.21s	remaining: 19.7s
176:	learn: 5039.234

309:	learn: 4817.1171261	total: 7.39s	remaining: 16.4s
310:	learn: 4815.8435048	total: 7.42s	remaining: 16.4s
311:	learn: 4814.5683363	total: 7.44s	remaining: 16.4s
312:	learn: 4812.3842295	total: 7.47s	remaining: 16.4s
313:	learn: 4811.2965862	total: 7.49s	remaining: 16.4s
314:	learn: 4809.0598093	total: 7.51s	remaining: 16.3s
315:	learn: 4808.0108485	total: 7.53s	remaining: 16.3s
316:	learn: 4807.4028284	total: 7.55s	remaining: 16.3s
317:	learn: 4805.2643670	total: 7.57s	remaining: 16.2s
318:	learn: 4804.0114421	total: 7.6s	remaining: 16.2s
319:	learn: 4802.8532757	total: 7.62s	remaining: 16.2s
320:	learn: 4802.0814387	total: 7.64s	remaining: 16.2s
321:	learn: 4801.5339575	total: 7.66s	remaining: 16.1s
322:	learn: 4799.6660475	total: 7.68s	remaining: 16.1s
323:	learn: 4798.6591464	total: 7.71s	remaining: 16.1s
324:	learn: 4797.9681727	total: 7.73s	remaining: 16.1s
325:	learn: 4796.6568106	total: 7.75s	remaining: 16s
326:	learn: 4795.6497944	total: 7.77s	remaining: 16s
327:	learn: 479

461:	learn: 4652.6728913	total: 11s	remaining: 12.8s
462:	learn: 4651.5889301	total: 11s	remaining: 12.8s
463:	learn: 4650.4391272	total: 11s	remaining: 12.7s
464:	learn: 4649.1753305	total: 11.1s	remaining: 12.7s
465:	learn: 4647.5531370	total: 11.1s	remaining: 12.7s
466:	learn: 4646.5373626	total: 11.1s	remaining: 12.7s
467:	learn: 4645.9048732	total: 11.1s	remaining: 12.6s
468:	learn: 4645.0522185	total: 11.1s	remaining: 12.6s
469:	learn: 4643.5397629	total: 11.2s	remaining: 12.6s
470:	learn: 4642.7239667	total: 11.2s	remaining: 12.6s
471:	learn: 4641.8894889	total: 11.2s	remaining: 12.5s
472:	learn: 4641.1937470	total: 11.2s	remaining: 12.5s
473:	learn: 4640.4655731	total: 11.3s	remaining: 12.5s
474:	learn: 4639.4013472	total: 11.3s	remaining: 12.5s
475:	learn: 4637.8667169	total: 11.3s	remaining: 12.4s
476:	learn: 4636.2617330	total: 11.3s	remaining: 12.4s
477:	learn: 4634.9293250	total: 11.3s	remaining: 12.4s
478:	learn: 4634.0366898	total: 11.4s	remaining: 12.4s
479:	learn: 4633

619:	learn: 4505.8165285	total: 14.6s	remaining: 8.97s
620:	learn: 4504.9381931	total: 14.7s	remaining: 8.95s
621:	learn: 4504.6403828	total: 14.7s	remaining: 8.92s
622:	learn: 4503.4628203	total: 14.7s	remaining: 8.9s
623:	learn: 4502.9940041	total: 14.7s	remaining: 8.87s
624:	learn: 4502.4095402	total: 14.7s	remaining: 8.85s
625:	learn: 4501.4642299	total: 14.8s	remaining: 8.82s
626:	learn: 4499.8447938	total: 14.8s	remaining: 8.8s
627:	learn: 4498.9921887	total: 14.8s	remaining: 8.77s
628:	learn: 4498.3941256	total: 14.8s	remaining: 8.75s
629:	learn: 4497.2362359	total: 14.9s	remaining: 8.72s
630:	learn: 4496.1425002	total: 14.9s	remaining: 8.7s
631:	learn: 4495.4364165	total: 14.9s	remaining: 8.67s
632:	learn: 4494.5968939	total: 14.9s	remaining: 8.65s
633:	learn: 4493.8101843	total: 14.9s	remaining: 8.62s
634:	learn: 4493.4696110	total: 15s	remaining: 8.6s
635:	learn: 4492.7466613	total: 15s	remaining: 8.57s
636:	learn: 4492.0323820	total: 15s	remaining: 8.55s
637:	learn: 4491.301

774:	learn: 4395.1133920	total: 18.3s	remaining: 5.31s
775:	learn: 4394.8032847	total: 18.3s	remaining: 5.28s
776:	learn: 4394.2709964	total: 18.3s	remaining: 5.26s
777:	learn: 4393.1613443	total: 18.3s	remaining: 5.23s
778:	learn: 4392.5514097	total: 18.4s	remaining: 5.21s
779:	learn: 4391.2200734	total: 18.4s	remaining: 5.19s
780:	learn: 4390.1671350	total: 18.4s	remaining: 5.17s
781:	learn: 4389.7927523	total: 18.4s	remaining: 5.14s
782:	learn: 4389.1199710	total: 18.5s	remaining: 5.12s
783:	learn: 4388.6286472	total: 18.5s	remaining: 5.1s
784:	learn: 4388.1201776	total: 18.5s	remaining: 5.08s
785:	learn: 4387.2128471	total: 18.6s	remaining: 5.05s
786:	learn: 4386.8252158	total: 18.6s	remaining: 5.03s
787:	learn: 4386.2434208	total: 18.6s	remaining: 5s
788:	learn: 4385.7763913	total: 18.6s	remaining: 4.98s
789:	learn: 4385.1324529	total: 18.7s	remaining: 4.96s
790:	learn: 4384.5267405	total: 18.7s	remaining: 4.93s
791:	learn: 4383.0780977	total: 18.7s	remaining: 4.91s
792:	learn: 43

930:	learn: 4298.5589596	total: 21.8s	remaining: 1.62s
931:	learn: 4298.1618715	total: 21.9s	remaining: 1.6s
932:	learn: 4297.4609818	total: 21.9s	remaining: 1.57s
933:	learn: 4296.4704971	total: 21.9s	remaining: 1.55s
934:	learn: 4295.6426085	total: 21.9s	remaining: 1.52s
935:	learn: 4294.9656664	total: 22s	remaining: 1.5s
936:	learn: 4294.5659065	total: 22s	remaining: 1.48s
937:	learn: 4293.8810945	total: 22s	remaining: 1.46s
938:	learn: 4293.3306701	total: 22s	remaining: 1.43s
939:	learn: 4292.8628490	total: 22.1s	remaining: 1.41s
940:	learn: 4292.4798161	total: 22.1s	remaining: 1.39s
941:	learn: 4291.9450409	total: 22.1s	remaining: 1.36s
942:	learn: 4291.1954679	total: 22.1s	remaining: 1.34s
943:	learn: 4290.6939697	total: 22.2s	remaining: 1.31s
944:	learn: 4289.5847035	total: 22.2s	remaining: 1.29s
945:	learn: 4289.0872968	total: 22.2s	remaining: 1.27s
946:	learn: 4288.3541731	total: 22.2s	remaining: 1.24s
947:	learn: 4287.8164317	total: 22.3s	remaining: 1.22s
948:	learn: 4287.445

In [32]:
mlrmodel=pd.Series({'Model Name':'Tuned CatBoost','Train R2 Score':r2score_train,'Test R2 Score':r2score_test,
                    'Train RMSE': rmse_train,'Test RMSE': rmse_test,'MAPE':mape})

result=result.append(mlrmodel,ignore_index=True)

result

,Model Name,Train R2 Score,Test R2 Score,Train RMSE,Test RMSE,MAPE
0,Decision Tree,1.000000,0.698599,0.000000,6791.589022,41.931885
1,Ridge,0.551583,0.550598,8324.314912,8293.102361,65.793066
2,Lasso,0.551568,0.550605,8324.456660,8293.032234,65.761432
3,Elastic Net,0.420095,0.421313,9466.420771,9410.684276,62.153973
4,Random Forest,0.978810,0.852304,1809.561929,4754.267106,43.121343
5,AdaBoost,0.631736,0.632103,7543.748157,7503.471796,60.270101
6,Gradient Boost,0.782004,0.784203,5804.057717,5746.743508,50.892615
7,XGBoost,0.860786,0.840426,4638.193758,4941.745539,42.187473
8,CatBoost,0.851805,0.840915,4785.464446,4934.172983,43.649447
9,Tuned CatBoost,0.882695,0.852174,4257.599924,4756.355426,40.828878


##### From above table we can conclude that our Tuned catboost model is final model.